# Ручка 3. Summary

Делает краткое саммари всего конспекта (на вход идет весь файл в обсидиане, можно добавить передачу всей папки)

In [64]:
from langchain.chat_models.gigachat import GigaChat
from dotenv import load_dotenv
import os

load_dotenv()

GIGACHAT_CREDENTIALS = os.environ.get("GIGACHAT_CREDENTIALS")

llm = GigaChat(
    credentials=GIGACHAT_CREDENTIALS,
    model="GigaChat-Pro",
    verify_ssl_certs=True,
    scope="GIGACHAT_API_CORP",
)

In [51]:
# Загружаем конспект
from langchain_community.document_loaders.text import TextLoader

notes_path = "./notes/habr.md"

loader = TextLoader(notes_path, "utf-8")
docs = loader.load()

print(docs[0].page_content)

# Используем Stuff Documents Chain + Summarization Checker

In [63]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMSummarizationCheckerChain

def stuff_summary(docs, checker: bool=False, max_checks:int=2):
     prompt_template = """Выдели клюбчевые моменты в тексте.
     Текст: "{context}"
     Ключевые моменты:"""
     prompt = PromptTemplate.from_template(prompt_template)
     stuff_chain = create_stuff_documents_chain(llm, prompt)
     res = stuff_chain.invoke({"context": docs})
     if not checker:
          return res
     else:
          checker_chain = LLMSummarizationCheckerChain.from_llm(llm, max_checks=max_checks)
          return checker_chain.run(res)

print(stuff_summary(docs, True, 1))

1. Количество неисправленных ошибок у SSD зависит от времени жизни диска, тогда как у HDD - от времени позиционирования головки диска (Head Flying Hours).
2. Нет научных исследований или доказательств, подтверждающих резкое увеличение количества ошибок HDD за 2 дня перед отказом.
3. В некоторых случаях на SSD может наблюдаться больше неисправленных ошибок по сравнению с HDD, особенно при длительном использовании.
4. Хотя накопители часто оснащены кодами коррекции ошибок, они могут не справиться со всеми видами ошибок, и полная надежность на них не всегда может быть возложена.


# Используем Map-Reduce

In [65]:
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter


# Map
map_template = """Ниже представлен список документов
{docs}
На основе этих документов определи, пожалуйста, основные темы 
Полезный ответ:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

reduce_template = """Ниже представлен набор кратких изложений:
{docs}
Возьми их и сведи в окончательное, объединенное краткое изложение основных тем. 
Полезный ответ:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=8000,
)

# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

map_reduce_chain.run(split_docs)

'Основные темы, которые можно выделить на основе представленных документов, включают:\n\n1. Отказоустойчивость приложений: важность обеспечения надежности и устойчивости программного обеспечения к сбоям и ошибкам.\n2. Работа с диском: изучение процесса записи данных на диск, включая буферизацию и её влияние на производительность и надежность.\n3. Языки программирования и их интерфейсы для работы с файлами: обзор различных языков программирования и их стандартных библиотек для работы с файлами, включая запись и чтение данных.\n4. Буферизация: преимущества и недостатки буферизации при работе с файлами, а также возможные проблемы, связанные с не сброшенными буферами.\n5. Управление буферами: необходимость контроля за сбросом буферов при завершении работы с файлами, чтобы избежать потери данных.\n6. Ошибки и сбои: рассмотрение ситуаций, когда приложение может быть закрыто внезапно (например, при получении сигнала SIGKILL), и как это может повлиять на сохранность данных в буферах.'